In [0]:
%fs ls /Volumes/workspace/default/emp/cust_address_json.csv

In [0]:
df_json = spark.read.format("csv").option("inferSchema",True).option("header",True).load('/Volumes/workspace/default/emp/cust_address_json.csv')
df_json.printSchema()

In [0]:
df_col = df_json.columns
print(df_col)
df_sel = df_json.select(df_col[:])
display(df_sel)

**EXPLODE**

In [0]:

# Sample data
data = [(1, "Jayesh", "Tendulkar", 101, ['SQL','Data Science','PySpark']),
        (2, "Rohit", "Sharma", 102, ['Data Analytics','ML','AI']),
        (3, "Sai", "Ramesh", 101, ['SSMS','Azure','AWS','DEVOPS']),
        (4, "Sreedhar", "Arava", 102, ['Database','Oracle','ADF']),
        (5, "Somesh", "yadav", 101, ['SQL','Data Science','GitHub','PANDAS']),
        (6, "Radhika", "Gupta", 102, ['DEVOPS','AWS','SSMS','Python'])
       ]

columns = ["emp_id", "first_name", "last_name", "dept_id", "Technology"]

# Create DataFrame
df = spark.createDataFrame(data, columns)
display(df)

# display data types
df.printSchema()

In [0]:
from pyspark.sql.functions import *
df_exp = df.select("emp_id","first_name","last_name","dept_id",explode("Technology").alias("coretech"))
df_exp.display()

In [0]:
data = [("Revanth", [["ADF", "Spark", "ADB"], ["ETL", "Devops", None], ["SQL", None]]),
        ("Reshma", [["SSMS", None, "Salesforce"], ["SAP", "ERP", None]]),
        ("Raashi", [["Python" "VB", None], ["C++", "GitHub", "Git"]]),
        ("Krishna", [["SHELL", "DRG"], ["JAVA", None]]),
        ("Sudarshan", None),
        ("Kamal", [])
       ]

columns = ["EmpName", "Technology"]

df_nest = spark.createDataFrame(data=data, schema=columns)
display(df_nest)
df_nest.printSchema()
     

In [0]:
df_nexp = df_nest.select("EmpName",explode("Technology").alias("coretech"))
df_flat = df_nexp.select("EmpName",explode("coretech").alias("parsed_array"))
df_nexp.display() 
df_flat.display()                     

**Error**

In [0]:
from pyspark.sql.functions import col

# Sample data for df1 and df2
data1 = [(1, "Alice", "123456"), (2, "Bob", "789012")]
data2 = [(1, "12345", "123 Main St", "CityA"), (2, "67890", "456 Elm St", "CityB")]

columns1 = ["id", "name", "mobno"]
columns2 = ["id", "pincode", "address", "city"]

df1 = spark.createDataFrame(data1, columns1)
display(df1)

df2 = spark.createDataFrame(data2, columns2)
display(df2)
     

In [0]:
df_join = df1.join(df2, df1.id == df2.id, "inner")
display(df_join)

In [0]:
df_jsel = df_json.select(df1.id,"name","mobno","pincode","address","city")
df_jsel.display()


In [0]:
df_jsel = df1.withColumnRenamed("id","df1_id").select("df1_id","name","mobno","pincode","address","city")
df_jsel.display()


In [0]:
Q1) How to find null rows in columns using pyspark?
 
 Q2) How do you clean below data using databricks. Handling blank and null?

         name         email        phoneNo   country
         xyz          blank         1234      null
         null    abs@gmail.com      blank     blank
         abc          null          null      india
         def     xyz@gmail.com      5678      sweden
         None         None          None      None

 # Sample data
 

In [0]:
data = [
     ("xyz", "blank", "1234", None),
     (None, "abs@gmail.com", "blank", "blank"),
     ("abc", None, None, "india"),
     ("def", "xyz@gmail.com", "5678", "sweden"),
     (None, None, None, None)]

schema = ["name","email","phno","country"]

df = spark.createDataFrame(data, schema)
display(df)

In [0]:
df_null = df.filter(df.name.isNull() | df.email.isNull() | df.phno.isNull() | df.country.isNull())
display(df_null)
df_clean = df.filter(df.name.isNotNull() & df.email.isNotNull() & df.phno.isNotNull() & df.country.isNotNull())
display(df_clean)

In [0]:
df_fill = df.fillna({"name" : "alpha", "email" : "abc@email.com", "phno": "67999", "country": "USA"})
df_fill.display()

In [0]:
df_all_null = df.filter(df_null.name.isNull() & df_null.email.isNull() & df_null.phno.isNull() & df_null.country.isNull())
df_all_null.drop.display()

                             

In [0]:
df_cole = df_null.dropna(how ='any', subset = (["name","email","phno","country"]))
df_cole.display()

In [0]:
# i) How to read above json using pyspark?
#  ii) How to get the `email` from `contacts` from above json using same code?

In [0]:
# Sample JSON data
json_data = '''
{
  "id": 1,
  "name": "John",
  "details": {
    "age": 25,
    "gender": "Male",
    "address": {
      "street": "123 Main St",
      "city": "Anytown",
      "state": "CA",
      "zip": "12345"
    },
    "contacts": {
      "email": "john@example.com",
      "phone": "+1-555-123-4567"
    }
  }
}
'''

dbutils.fs.put('/Volumes/workspace/default/emp/json_data.json', json_data, True)
#df = spark.read.json('/Volumes/workspace/default/emp/json_data.json')

               


In [0]:
df_json = spark.read.format('json')\
                    .option('header', True)\
                    .option('inferSchema', True)\
                    .option('multiLine', True)\
                    .load("/Volumes/workspace/default/emp/json_data.json")
display(df_json)
     

In [0]:
# Extract the email from contacts
email_df = df_json.select("id","name",col("details.age").alias("age"),col("details.gender").alias("gender"),col("details.address.city").alias("city"),col("details.address.state").alias("state"),col("details.address.zip").alias("zip"),col("details.contacts.email").alias("email"),col("details.contacts.phone").alias("phone"))

# Display the email DataFrame
display(email_df)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df_jsonp = df_json.select(
    col("details.address.city").alias("city"),
    col("details.address.state").alias("state"),
    col("details.address.street").alias("street"),
    col("details.address.zip").alias("zip")
)
display(df_jsonp)

In [0]:
data = [
    (1, ["a", "b", "c"], "d")
]

# Define schema
columns = ["item_id", "c1", "c2"]

# Create DataFrame
df_Q4 = spark.createDataFrame(data, columns)
display(df_Q4)
     

In [0]:
df_p = df_Q4.withColumn("c1_new", df_Q4.c1[0]).alias("c1").withColumn("c2_new", df_Q4.c1[1]).alias("c2").withColumn("c3_new", df_Q4.c1[2]).alias("c3")
display(df_p)

In [0]:
schema = ["Name", "SkillSet"]
    
data = (["ABC", ['.Net', 'Git', 'C#']],
        ["XYZ", ['Wordpress', 'PHP']],
        ["IJK", ['Python', 'MongoDB', 'Git']],
        ["DEF", ['SSIS', 'SSAS', 'Power BI', 'SQL Server', 'Data Warehouse']],
        ["PQR", ['Azure']])

df = spark.createDataFrame(data, schema)
df.show(truncate=False)
df.printSchema()

In [0]:
for i in range(5):
    df = df.withColumn(f"skill_{i}", df.SkillSet.getItem(i))
    
df.show()

In [0]:
schema = ["Name", "SkillSet"]
    
data = (["ABC", ['.Net', 'Git', 'C#']],
        ["XYZ", ['Wordpress', 'PHP']],
        ["IJK", ['Python', 'MongoDB', 'Git']],
        ["DEF", ['SSIS', 'SSAS', 'Power BI', 'SQL Server', 'Data Warehouse']],
        ["PQR", ['Azure']])

df = spark.createDataFrame(data, schema)
df.show(truncate=False)
df.printSchema()

In [0]:

d21 = df.withColumn("Skill1", df['SkillSet'][0]) \
        .withColumn("Skill2", df['SkillSet'][1]) \
        .withColumn("Skill3", df['SkillSet'][2]) \
        .withColumn("Skill4", df['SkillSet'][3]) \
        .withColumn("Skill5", df['SkillSet'][4])
d21.show(truncate=False)

In [0]:

dfsize = df.select("Name", "SkillSet", size("SkillSet").alias("NoOfArrayElements"))
dfsize.show(truncate=False)

In [0]:
max_val = dfsize.select(max("NoOfArrayElements")).collect()[0][0]
print(max_val)


In [0]:
def maxarray(df, maxval):
    for i in range(maxval):
        df = df.withColumn(f"skill_{i}", df['SkillSet'][i])
    return df

In [0]:
df_out = maxarray(df, max_val)
df.show(truncate=False)
                

In [0]:

df.select('Name', 'SkillSet', df.SkillSet[0].alias('Skill1'),\
                              df.SkillSet[1].alias('Skill2'),\
                              df.SkillSet[2].alias('Skill3'),\
                              df.SkillSet[3].alias('Skill4'),\
                              df.SkillSet[4].alias('Skill5')).show()

In [0]:
df_split = spark.read.format("csv").option("header",True).option("inferSchema",True).load("/Volumes/workspace/default/emp/SPLIT_PY.csv")

df_split.display()

In [0]:
df_new = df_split.withColumn("new_column", split("column2",'\|')[0]).withColumn("new_col2",split("column2",'\|')[1])
df_new.display()
df_nnew = df_new.withColumn("new_column1", split("new_column", '-')[0])
df_nnew.display()
#df_new = df_split.withColumn("new_column", split("column2",'\|')[0])

In [0]:

# Initialize two lists
subjects1 = ["Java","Python","PHP","SQL"]
subjects2 = ['C#','CPP','C','plsql']

final = list(zip(subjects1,subjects2))
print(final)

In [0]:

from pyspark.sql.functions import locate

# Sample data
data = [("Azure data engineer (ADE)", "suman@gmail.com"),
        ("AWS data engineer (AWS)", "kiranrathod@gmail.com"),
        ("data warehouse", "rameshwaran@gmail.com"),
        ("GCP engineer", "krishnamurthy@gmail.com"),
        ("PySpark engineer", "vishweswarrao@gmail.com")]

columns = ["text", "mail"]

df = spark.createDataFrame(data, columns)
display(df)

In [0]:
df_exp = df.withColumn("position", locate("data", df.text))
df_exp.show()
df_exp.filter(df_exp.position > 0).show()

df_email = df.withColumn("email_pos",locate("@", df.mail)).display()

In [0]:
df_em = df_email.withColumn(
    "new_mail",
    substring(df_email.mail, df_email.email_pos + 1, length(df_email.mail) - df_email.email_pos)
)

# Display the DataFrame
display(df_em)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
#df_em = df_email.select("text", "mail", "email_pos")
df_ex = withColumn("newcval",expr('substring("mail",1,locate("@",mail)) '))

In [0]:
df_ex = df_email.withColumn(
    "newcval",
    expr('substring("mail", 1, instr("mail", "@"))')

In [0]:
df_em = df_email.withColumn("new_mail", substring(df.mail, df.email_pos+1, length(df.mail)-df.email_pos)).display()